ML Phases

## Data Preprocessing

### Importing library

In [2]:
pip install tensorflow

  Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.0-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.4-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (

In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import re
import numpy as np
import os

ImportError: Traceback (most recent call last):
  File "c:\Users\Chinnu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 70, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

### Data Importing

In [2]:
data  = open('/content/drive/MyDrive/ML Docs/ML Projects/next-word-prediction/data/nextWordPrediction.txt', 'r', encoding = "utf8")

### EDA (evaluation Data Analysis)

### missing values

### Feature Engineering

In [ ]:
lines=[]
for i in data:
    lines.append(i)

In [4]:
data=""
for i in lines:
    data = ' '.join(lines)
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [5]:
# Remove punctuation and special characters
# data = re.sub(r'[^\w\s]', '', data)

In [6]:
data = data.split()
data=' '.join(data)

# Convert text to lowercase
data = data.lower()

In [7]:
data[:1000]

"project gutenberg's the adventures of sherlock holmes, by arthur conan doyle this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.net title: the adventures of sherlock holmes author: arthur conan doyle release date: november 29, 2002 [ebook #1661] last updated: may 20, 2019 language: english character set encoding: utf-8 *** start of this project gutenberg ebook the adventures of sherlock holmes *** produced by an anonymous project gutenberg volunteer and jose menendez cover the adventures of sherlock holmes by arthur conan doyle contents i. a scandal in bohemia ii. the red-headed league iii. a case of identity iv. the boscombe valley mystery v. the five orange pips vi. the man with the twisted lip vii. the adventure of the blue carbuncle viii. the adventure of the speckled band ix. the adventure 

### Tokenizer Initialization Training

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# Save the tokenizer to a file
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequences = tokenizer.texts_to_sequences([data])[0]

In [9]:
# len(sequences)

In [10]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

8624

In [11]:
sequence_length = 5  # Set the length of your input sequence
input_sequences = []
output_words = []

for i in range(sequence_length, len(sequences)):
    input_sequences.append(sequences[i-sequence_length:i])
    output_words.append(sequences[i])

input_sequences = np.array(input_sequences)
output_words = np.array(output_words)


In [12]:
from keras.preprocessing.sequence import pad_sequences
max_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')


### Handling outliers

### Data Visualization

## Model Building

### Train and split data

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(input_sequences, output_words, test_size=0.2, random_state=42)


### Model initialization

In [14]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam


In [15]:
model = Sequential()


In [16]:
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size from the tokenizer
embedding_dim = 100  # You can adjust this value based on your preference

model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))  # Softmax activation for multi-class classification


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Model Training

In [18]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)


Epoch 1/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 68s 48ms/step - accuracy: 0.0532 - loss: 6.9604 - val_accuracy: 0.0584 - val_loss: 6.5705
Epoch 2/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 89s 54ms/step - accuracy: 0.0638 - loss: 6.3638 - val_accuracy: 0.0634 - val_loss: 6.5827
Epoch 3/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 77s 50ms/step - accuracy: 0.0740 - loss: 6.1081 - val_accuracy: 0.0699 - val_loss: 6.6176
Epoch 4/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 80s 49ms/step - accuracy: 0.0904 - loss: 5.7695 - val_accuracy: 0.0720 - val_loss: 6.7875
Epoch 5/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 82s 49ms/step - accuracy: 0.1083 - loss: 5.4411 - val_accuracy: 0.0706 - val_loss: 7.0881
Epoch 6/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 83s 50ms/step - accuracy: 0.1317 - loss: 5.1118 - val_accuracy: 0.0648 - val_loss: 7.4190
Epoch 7/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 80s 48ms/step - accuracy: 0.1616 - loss: 4.7513 - val_accuracy: 0.0616 - val_loss: 7.9185
Epoch 8/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 82s 49ms/step - accuracy: 0.1984 -

### Hyper Parameter Tuning

## Evaluation and Result

In [19]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


681/681 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.0502 - loss: 10.9124
Validation Accuracy: 4.91%


## Using the Model to Predict the Next Word

In [20]:
def predict_next_word(model, tokenizer, text_seq, max_len):
    sequence = tokenizer.texts_to_sequences([text_seq])[0]
    sequence = pad_sequences([sequence], maxlen=max_len, padding='pre')
    predicted = model.predict(sequence)
    predicted_word = tokenizer.index_word[np.argmax(predicted)]
    return predicted_word

text_seq = "The Adventures of"
next_word = predict_next_word(model, tokenizer, text_seq, max_len)
print(f"Next word prediction: {next_word}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Next word prediction: sherlock
